In [1]:
import sys
sys.path.insert(0, '../../../network')

import logging
import argparse
import itertools

import numpy as np
from scipy.stats import norm
from numba import jit, njit

from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

Load data from figure 1

In [3]:
datapath = "../1/data/data.npy"
state = np.load(datapath, allow_pickle=True)[3]

Compute peak information

In [9]:
@jit
def func(state):
    all_peaks = [[]]*40000
    for n, row in enumerate(state):
        thresh = row.mean()
        sigma = row.std()
        idxs = row > thresh + 1*sigma
        peaks = [np.array([])]
        peak_idx = 0
        prev_value = False
        for m, i in enumerate(idxs):
            if i:
                if prev_value == False and i:
                    peaks.append(np.array([n]))
                    peak_idx += 1
                else:
                    peaks[peak_idx] = np.r_[peaks[peak_idx], m]
            prev_value = i
        all_peaks[n] = peaks
    return all_peaks

peaks = func(state)

<ipython-input-9-34d18e1f4455>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "func" failed type inference due to: Undecided type $118load_method.3 := <undecided>
[1] During: resolving caller type: $118load_method.3
[2] During: typing of call at <ipython-input-9-34d18e1f4455> (14)


File "<ipython-input-9-34d18e1f4455>", line 14:
def func(state):
    <source elided>
                if prev_value == False and i:
                    peaks.append(np.array([n]))
                    ^

  @jit
INFO:numba.transforms:finding looplift candidates
<ipython-input-9-34d18e1f4455>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "func" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-9-34d18e1f4455>", line 4:
def func(state):
    <source elided>
    all_peaks = [[]]*40000
    for n, row in enumerate(state):
    ^

  @

KeyboardInterrupt: 

In [12]:
np.save(open("data/data.npy", "wb"), [peaks])